# Saving and Loading Tutorial

First, you need to have `Python3` and `openmpi` installed and running on your machine.

We'll make a virtual environment first...



## Normal saving

Then we'll run a simulation with normal saving:

In [ ]:
!mpiexec -n 4 nrniv -python -mpi saving_normal_init.py

This does not exit to the system prompt, so you will have to restart your kernel.  

The `whos` in the next cell should return `Interactive namespace is empty.`

In [ ]:
whos

In [ ]:
from netpyne import sim
sim.loadAll('saving_normal_data/saving_normal_data.pkl')

In [ ]:
sim.analysis.plotConn(saveFig='saving_normal_data/saving_normal_plot_conn_pop_strength_matrix_FROMFILE.png');
sim.analysis.plotRaster(saveFig='saving_normal_data/saving_normal_raster_gid_FROMFILE.png');
sim.analysis.plotTraces(saveFig='saving_normal_data/saving_normal_traces_FROMFILE.png');

Compare the plots, they should be identical.

Now restart your kernel and check the `whos`.

In [ ]:
whos

## Distributed Saving

If you're running large sims, you may want to save the data from each node in a separate file, i.e. distributed saving.

In [ ]:
!mpiexec -n 4 nrniv -python -mpi saving_dist_init.py

Now restart your kernel so we can load the data from file.

The `whos` in the next cell should return `Interactive namespace is empty.`

In [ ]:
whos

In [ ]:
from netpyne import sim
sim.gatherDataFromFiles(simLabel='saving_dist')

In [ ]:
sim.analysis.plotConn(saveFig='saving_dist_data/saving_dist_plot_conn_pop_strength_matrix_FROMFILE.png');
sim.analysis.plotRaster(saveFig='saving_dist_data/saving_dist_raster_gid_FROMFILE.png');
sim.analysis.plotTraces(saveFig='saving_dist_data/saving_dist_traces_FROMFILE.png');

Compare the plots, they should be identical.

Now restart your kernel and check the `whos`.

In [ ]:
whos

## Interval Saving

Perhaps you want to save data at intervals in case you have large, long simulations you're worried won't complete.

In [ ]:
!mpiexec -n 4 nrniv -python -mpi saving_int_init.py

Now restart your kernel so we can load the data from file.

The `whos` in the next cell should return `Interactive namespace is empty.`

In [ ]:
whos

Now let's assume our simulation timed out, and the last interval save we got was at 5000 ms.

In [ ]:
from netpyne import sim
sim.loadAll('saving_int_data/interval_data/interval_5000.pkl', createNEURONObj=False)

In [ ]:
sim.analysis.plotConn(saveFig='saving_int_data/saving_int_plot_conn_pop_strength_matrix_INTERVAL.png');
sim.analysis.plotRaster(saveFig='saving_int_data/saving_int_raster_gid_INTERVAL.png');
sim.analysis.plotTraces(saveFig='saving_int_data/saving_int_traces_INTERVAL.png');